### SRI패널조사

In [44]:
import os
import json
import pandas as pd
from pathlib import Path
from pprint import pprint

In [45]:
# 디렉토리 및 파일 경로
OUTPUT_DIR_PATH = Path.cwd() / "output"

csv_path = OUTPUT_DIR_PATH / "csv" / "(공개용) 2024년 3분기 패널조사 데이터(가중치).csv"

In [58]:
import json

# 코드북 로드
with open(OUTPUT_DIR_PATH / "codebooks" / "2024년 3분기 패널조사 데이터_코드북.json", "r", encoding="utf-8") as f:
    codebook = json.load(f)

def what_is(column_name: str):
    """컬럼명에 해당하는 section, label, choices를 보기 좋게 출력"""
    variables = codebook.get("variables", {})
    col_info = variables.get(column_name)

    if not col_info:
        return f"'{column_name}' 컬럼은 코드북에 없음."

    section = col_info.get("section", "N/A")
    label = col_info.get("label", "N/A")
    choices = col_info.get("choices", None)

    output = f"[{section}] {column_name}\n"
    output += f"질문: {label}\n"

    if choices:
        output += "응답 선택지:\n"
        for c in choices:
            output += f"  - {c['code']}: {c['label']}\n"

    return output.strip()

# 사용 예시
print(what_is("q1"))


[Ⅰ. 수원시정 및 의회 만족도] q1
질문: 1. 귀하는 수원시의 시책이나 시정 운영에 대해 어느 정도 알고 계십니까?
응답 선택지:
  - 1: 전혀 모르고 있다
  - 4: 보통이다
  - 7: 매우 잘 알고 있다


In [46]:
# 개별 접근 예시
df_3q_2024 = pd.read_csv(csv_path)
df = df_3q_2024

In [47]:
# 전체 데이터 개요
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1784 entries, 0 to 1783
Columns: 137 entries, quarter to q9_5_op
dtypes: float64(70), int64(5), object(62)
memory usage: 1.9+ MB
None


In [48]:
# 전체 컬럼 조회
pprint(df.columns)

Index(['quarter', 'source_file', 'respondent_id', '_id', 'ID', 'GU_code',
       'GU_label', 'resi_y', 'resi_m', 'q1_code',
       ...
       'DM9_4_label', 'q14_op', 'DM12_op', 'q16_op', 'q15_op', 'q9_2_op',
       'q3_3_16_op', 'q17_op', 'q6_op', 'q9_5_op'],
      dtype='object', length=137)


In [49]:
# 불필요한 컬럼 제거
df = df.drop(columns=["quarter", "source_file", "_id"])
df.head()

,respondent_id,ID,GU_code,GU_label,resi_y,resi_m,q1_code,q2_code,q3_code,q3_1_1_code,...,DM9_4_label,q14_op,DM12_op,q16_op,q15_op,q9_2_op,q3_3_16_op,q17_op,q6_op,q9_5_op
0,1,1,1,장안구,8.0,1.0,5.0,5.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,1,장안구,40.0,0.0,5.0,6.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,4,영통구,30.0,5.0,3.0,3.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,1,장안구,7.0,11.0,3.0,5.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,1,장안구,4.0,10.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
with open(OUTPUT_DIR_PATH / "codebooks" / "2024년 3분기 패널조사 데이터_코드북.json", "r", encoding="utf-8") as f:
    codebook = json.load(f)

# 코드북 로드
valid_columns = set(codebook["variables"].keys())

# 코드북에 없는 컬럼 제거
for col in df.columns:
    if col not in valid_columns:
        df.drop(columns=[col], inplace=True)

print(len(df.columns))
print(df.head())

12
   ID  resi_y  resi_m  q13_2  DM2  DM5  DM8      wg      ws   q9_4 q21_op  \
0   1     8.0     1.0  100.0   39  4.0  2.0  267.21  0.4692    NaN    NaN   
1   2    40.0     0.0   50.0   40  3.0  1.0  286.97  0.5039  200.0    NaN   
2   3    30.0     5.0  100.0   30  4.0  NaN  216.81  0.3807   30.0    NaN   
3   4     7.0    11.0    NaN   27  4.0  NaN  276.17  0.4849   40.0    NaN   
4   5     4.0    10.0  150.0   34  4.0  2.0  198.29  0.3482    NaN    NaN   

  q10_op  
0    NaN  
1    NaN  
2    NaN  
3    NaN  
4    NaN  


In [52]:
# 결측치 확인
df_3q_2024 = df_3q_2024.dropna(how='all')   # 모든 컬럼이 NaN인 행 삭제
print(df_3q_2024.isna().sum().to_string())
df_3q_2024 = df_3q_2024.drop(columns=['q3_3_16_op', 'q17_op', 'q6_op', 'q9_5_op'])    # 열린 질문 컬럼 제거

quarter             0
source_file         0
respondent_id       0
_id                 0
ID                  0
GU_code             0
GU_label            0
resi_y              3
resi_m              4
q1_code             1
q2_code             1
q3_code             1
q3_1_1_code       778
q3_1_1_label      778
q3_1_2_code       778
q3_1_2_label      778
q3_1_3_code       778
q3_1_3_label      778
q4_code             3
q4_label            3
q5_1_code           3
q5_2_code           3
q5_3_code           3
q5_4_code           3
q5_5_code           3
q6_code             3
q6_label            3
q7_code             3
q8_code             3
q8_label            3
q9_code             3
q9_label            3
q10_code          987
q10_label         987
q11_1_code          3
q11_1_label         3
q11_2_code          3
q11_2_label         3
q12_code            4
q12_label           4
q13_code            5
q13_label           5
q13_1_code        849
q13_1_label       849
q13_2             851
q14_code  

In [51]:
cols_to_drop = [
    'q14_op', 'DM12_op', 'q16_op', 'q15_op', 'q9_2_op',
    'q3_3_16_op', 'q17_op', 'q6_op', 'q9_5_op'
]
df.dropna(axis=1, how='all', inplace=True)  # df에 직접 적용
df = df.drop(columns=cols_to_drop)

print(len(df.columns))
df.head()


125


,respondent_id,ID,GU_code,GU_label,resi_y,resi_m,q1_code,q2_code,q3_code,q3_1_1_code,...,q5_4_label,q5_1_label,q1_label,q2_label,q21_op,q10_op,DM9_3_code,DM9_3_label,DM9_4_code,DM9_4_label
0,1,1,1,장안구,8.0,1.0,5.0,5.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,1,장안구,40.0,0.0,5.0,6.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,4,영통구,30.0,5.0,3.0,3.0,4.0,NaN,...,보통이다,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,1,장안구,7.0,11.0,3.0,5.0,4.0,NaN,...,NaN,보통이다,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,1,장안구,4.0,10.0,4.0,4.0,4.0,NaN,...,보통이다,NaN,보통이다,보통이다,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_3q_2024.dropna(how='all')

      respondent_id    ID  GU_code GU_label  resi_y  resi_m  q1_code  q2_code  \
0                 1     1        1      장안구     8.0     1.0      5.0      5.0   
1                 2     2        1      장안구    40.0     0.0      5.0      6.0   
2                 3     3        4      영통구    30.0     5.0      3.0      3.0   
3                 4     4        1      장안구     7.0    11.0      3.0      5.0   
4                 5     5        1      장안구     4.0    10.0      4.0      4.0   
...             ...   ...      ...      ...     ...     ...      ...      ...   
1779           1780  1780        1      장안구     4.0     0.0      5.0      5.0   
1780           1781  1781        4      영통구    33.0     7.0      5.0      6.0   
1781           1782  1782        2      권선구     7.0     3.0      5.0      4.0   
1782           1783  1783        3      팔달구     3.0     4.0      4.0      2.0   
1783           1784  1784        2      권선구    26.0    11.0      6.0      5.0   

      q3_code  q3_1_1_code 

In [1]:
# [q1] 1. 귀하는 수원시의 시책이나 시정 운영에 대해 어느 정도 알고 계십니까
all_quarters_path = OUTPUT_DIR_PATH / "jsonl" / "(공개용) 2024년 3분기 패널조사 데이터(가중치).jsonl"

df = pd.read_json(all_quarters_path, lines=True)

print(df.iloc[0])


NameError: name 'OUTPUT_DIR_PATH' is not defined